### Step 1. Set up

In [1]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

### Step 2. Load the Keras model using the json and weights file

In [2]:
import keras
from keras.models import model_from_json

Using TensorFlow backend.


Create a directory called keras_model, navigate to keras_model from the Jupyter notebook home, and upload the model.json and model-weights.h5 files

In [3]:
!mkdir keras_model

In [4]:
!ls keras_model

model.json  model-weights.h5


In [5]:
json_file = open('/home/ec2-user/SageMaker/keras_model/'+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [6]:
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/model-weights.h5')
print("Loaded model from disk")

Loaded model from disk


### Step 3. Export the Keras model to the TensorFlow ProtoBuf format

In [7]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [8]:
# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [9]:
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

In [10]:
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

In [11]:
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

### Step 4. Convert TensorFlow model to a SageMaker readable format

Move the TensorFlow exported model into a directory export\Servo\. SageMaker will recognize this as a loadable TensorFlow model. Your directory and file structure should look like:

In [12]:
!ls export

Servo


In [13]:
!ls export/Servo

1


In [14]:
!ls export/Servo/1

saved_model.pb	variables


In [15]:
!ls export/Servo/1/variables

variables.data-00000-of-00001  variables.index


####  Tar the entire directory and upload to S3

In [16]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [17]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-2-471064563256


### Step 5. Deploy the trained model

The entry_point file "train.py" can be an empty Python file. The requirement will be removed at a later date.

In [18]:
!touch train.py

In [19]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

In [20]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-2019-03-06-19-10-50-938
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-2019-03-06-19-10-50-938


--------------------------------------------------------------!CPU times: user 462 ms, sys: 36.8 ms, total: 499 ms
Wall time: 5min 14s


Note: You will need to update the endpoint in the command below with the endpoint name from the output of the previous cell (INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-2018-09-28-14-26-05-826)

In [21]:
endpoint_name = 'sagemaker-tensorflow-2019-03-06-19-10-50-938'

In [22]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

### Step 6. Invoke the endpoint

In [25]:
! pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 14.1MB/s ta 0:00:01
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [27]:
! pip install imutils

  Stored in directory: /home/ec2-user/.cache/pip/wheels/b2/40/59/139d450e68847ef2f27d876d527b13389dac23df0f66526b5d
Successfully built imutils


In [28]:
! pip install opencv-python

    100% |████████████████████████████████| 25.4MB 1.8MB/s eta 0:00:01    65% |█████████████████████           | 16.7MB 25.1MB/s eta 0:00:01


In [29]:
import cv2
from keras.preprocessing.image import img_to_array
import imutils

In [30]:
def preprocess(image,scale=128):
    """ preprocess the test image and covert to array """
    inter=cv2.INTER_AREA
    (h, w) = image.shape[:2]
    dW = 0
    dH = 0
    
    width = scale
    height = scale

    # if the width is smaller than the height, then resize
    # along the width (i.e., the smaller dimension) and then
    # update the deltas to crop the height to the desired
    # dimension
    if w < h:
        image = imutils.resize(image, width=width,
            inter=inter)
        dH = int((image.shape[0] - height) / 2.0)

    # otherwise, the height is smaller than the width so
    # resize along the height and then update the deltas
    # crop along the width
    else:
        image = imutils.resize(image, height=height,
            inter=inter)
        dW = int((image.shape[1] - width) / 2.0)

    # now that our images have been resized, we need to
    # re-grab the width and height, followed by performing
    # the crop
    (h, w) = image.shape[:2]
    image = image[dH:h - dH, dW:w - dW]

    # finally, resize the image to the provided spatial
    # dimensions to ensure our output image is always a fixed
    # size
    
    image =  cv2.resize(image, (width, height),
        interpolation=inter)
    return img_to_array(image)

In [43]:
test1 = cv2.imread('/home/ec2-user/SageMaker/inputimage/dia.jpg')

In [44]:
indata = preprocess(test1, scale = 128)

In [45]:
indata.shape

(128, 128, 3)

#### Invoke the SageMaker endpoint from the notebook

In [46]:
predictor.predict(indata)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-tensorflow-2019-03-06-19-10-50-938 in account 471064563256 for more information.